In [1]:
green_areas = r'D:\osm2lulc01\uatlas\forest_and_evu.shp'

green_fid =  'ffif'

points_forest = r'D:\osm2lulc01\osm2lulc01.gdb\osm\helping_points_forest'

points_evu = r'D:\osm2lulc01\osm2lulc01.gdb\osm\helping_points_evu'

outfeat = r'D:\osm2lulc01\osm2lulc01.gdb\result_points'

tmpws = r'D:\osm2lulc01\osm2lulc01.gdb\corre'

In [2]:
import arcpy
import os

from gesri.dp import copy_feat
from gesri.rd.shp import shp_to_lyr
from gesri.gp.ovl import intersection
from gesri.gp.gen import dissolve
from gesri.tbl.joins import join_table
from gesri.tbl.col import rename_col
from gesri.tbl.col import cols_calc

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
srs = "PROJCS[\"ETRS_1989_Portugal_TM06\",GEOGCS[\"GCS_ETRS_1989\",DATUM[\"D_ETRS_1989\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-8.133108333333334],PARAMETER[\"Scale_Factor\",1.0],PARAMETER[\"Latitude_Of_Origin\",39.66825833333333],UNIT[\"Meter\",1.0]]"

In [5]:
cp_green = copy_feat(green_areas, outfeat)

glyr = shp_to_lyr(cp_green)

d = {
    'forest' : shp_to_lyr(points_forest),
    'evu'    : shp_to_lyr(points_evu)
}

In [6]:
for k in d:
    # Intersect
    oint = os.path.join(tmpws, f'i_{k}')
    ilyr = intersection([glyr, d[k]], oint)

    # Dissolve
    odiss = os.path.join(tmpws, f'diss_{k}')
    disslyr = dissolve(
        ilyr, odiss, green_fid,
        statistics=[[green_fid, 'COUNT']]
    )
    
    # Join
    green_lyr = join_table(
        cp_green, disslyr,
        green_fid, green_fid,
        cols=[f"COUNT_{green_fid}"]
    )
    
    # Rename column
    green_lyr = rename_col(green_lyr, f"COUNT_{green_fid}", f"pnts_{k}")
    
    # Fill null values
    green_lyr = cols_calc(green_lyr, f"pnts_{k}", f"fillna(!pnts_{k}!)", code="""def fillna(np):
        if not np:
            return 0
        
        return np"""
    )